In [ ]:
from sklearn import preprocessing
import http.client, urllib.parse
import pandas as pd
import numpy as np
import math
import json

In [ ]:
def getDist(houses, locations):
    
    min_dist = 999999999
    
    for house in houses:
        x1 = house['latitude']
        x2 = house['longitude']
        for location in locations:
            x2 = location['latitude']
            y2 = location['longitude']
            dist = (math.sqrt((x2 - x1)**2 + (y2 - y1)**2))*100
            if dist < min_dist:
                min_dist = dist
                
        distances.append(min_dist)  
    
    return distances

def getCoords(df):
        
    #api info
    conn = http.client.HTTPConnection('api.positionstack.com')
    key = 'f43f7a6a88da1f88685ee7c35dafb4f5'
        
        
    def query(coord, address):
               
        params = urllib.parse.urlencode({
                'access_key': key,
                'query': address,
                'limit': 1
            })

        conn.request('GET', '/v1/forward?{}'.format(params))
        try:
            results = conn.getresponse()
        except:
            query(coord,address)
            print("error")
        try:
            data = json.loads(results.read())['data'][0] #converts json to dict
            if coord == 'x':
                result = data['latitude']
            else:
                result = data['longitude']
            return result
        except:
            pass
            print("error")
        

    lat = [query('x', address) for address in df.iloc[:,0]]
    lon = [query('y', address) for address in df.iloc[:,0]]
        
    df['latitude'] = lat
    df['longitude'] = lon
        
    return df


In [ ]:
#IMPORT AND ORGANIZE DATA

houses = pd.read_csv('/data/houses_clean.csv')

houses_la = houses.subset(houses['City'] == 'Los Angeles'.tolower())
houses_anaheim = houses.subset(houses['City'] == 'Anaheim')
houses_longbeach = houses.subset(houses['City'] == 'Long Beach')

houses_dallas = houses.subset(houses['City'] == 'Dallas')
houses_fortworth = houses.subset(houses['City'] == 'Fort Worth')
houses_atx = houses.subset(houses['City'] == 'Arlington' and houses['State'] == 'TX')

houses_chicago = houses.subset(houses['City'] == 'Chicago')
houses_naperville = houses.subset(houses['City'] == 'Naperville')
houses_elgin = houses.subset(houses['City'] == 'Elgin')

houses_washington = houses.subset(houses['City'] == 'Washington')
houses_alex = houses.subset(houses['City'] == 'Alexandria')
houses_ava = houses.subset(houses['City'] == 'Arlington' and houses['State'] == 'VA')


schools = pd.read_csv('/data/schools_clean.cvs')
schools_la = schools.subset(schools['City'] == 'Los Angeles'.tolower())
schools_anaheim = schools.subset(schools['City'] == 'Anaheim')
schools_longbeach = schools.subset(schools['City'] == 'Long Beach')

schools_dallas = schools.subset(schools['City'] == 'Dallas')
schools_fortworth = schools.subset(schools['City'] == 'Fort Worth')
schools_atx = schools.subset(schools['City'] == 'Arlington' and schools['State'] == 'TX')

schools_chicago = schools.subset(schools['City'] == 'Chicago')
schools_naperville = schools.subset(schools['City'] == 'Naperville')
schools_elgin = schools.subset(schools['City'] == 'Elgin')

schools_washington = schools.subset(schools['City'] == 'Washington')
schools_alex = schools.subset(schools['City'] == 'Alexandria')
schools_ava = schools.subset(schools['City'] == 'Arlington' and schools['State'] == 'VA')


other = pd.read_csv('data/other_locations_clean.csv')
hospitals_la = other.subset(other['City'] == 'Los Angeles')['hospital']
hospitals_lb = other.subset(other['City'] == 'Long Beach')['hospital']
hospitals_a = other.subset(other['City'] == 'Anaheim')['hospital']

hospitals_d = other.subset(other['City'] == 'Dallas')['hospital']
hospitals_atx = other.subset(other['City'] == 'ArlingtonTX')['hospital']
hospitals_fw = other.subset(other['City'] == 'Fort Worth')['hospital']

hospitals_c = other.subset(other['City'] == 'Chicago')['hospital']
hospitals_e = other.subset(other['City'] == 'Elgin')['hospital']
hospitals_n = other.subset(other['City'] == 'Naperville')['hospital']

hospitals_dc = other.subset(other['City'] == 'Washington')['hospital']
hospitals_ava = other.subset(other['City'] == 'ArlingtonVA')['hospital']
hospitals_alex = other.subset(other['City'] == 'Alexandria')['hospital']

grocery_la = other.subset(other['City'] == 'Los Angeles')['grocery']
grocery_lb = other.subset(other['City'] == 'Long Beach')['grocery']
grocery_a = other.subset(other['City'] == 'Anaheim')['grocery']

grocery_d = other.subset(other['City'] == 'Dallas')['grocery']
grocery_atx = other.subset(other['City'] == 'ArlingtonTX')['grocery']
grocery_fw = other.subset(other['City'] == 'Fort Worth')['grocery']

grocery_c = other.subset(other['City'] == 'Chicago')['grocery']
grocery_e = other.subset(other['City'] == 'Elgin')['grocery']
grocery_n = other.subset(other['City'] == 'Naperville')['grocery']

grocery_dc = other.subset(other['City'] == 'Washington')['grocery']
grocery_ava = other.subset(other['City'] == 'ArlingtonVA')['grocery']
grocery_alex = other.subset(other['City'] == 'Alexandria')['grocery']

gym_la = other.subset(other['City'] == 'Los Angeles')['gym']
gym_lb = other.subset(other['City'] == 'Long Beach')['gym']
gym_a = other.subset(other['City'] == 'Anaheim')['gym']

gym_d = other.subset(other['City'] == 'Dallas')['gym']
gym_atx = other.subset(other['City'] == 'ArlingtonTX')['gym']
gym_fw = other.subset(other['City'] == 'Fort Worth')['gym']

gym_c = other.subset(other['City'] == 'Chicago')['gym']
gym_e = other.subset(other['City'] == 'Elgin')['gym']
gym_n = other.subset(other['City'] == 'Naperville')['gym']

gym_dc = other.subset(other['City'] == 'Washington')['gym']
gym_ava = other.subset(other['City'] == 'ArlingtonVA')['gym']
gym_alex = other.subset(other['City'] == 'Alexandria')['gym']

parks_la = other.subset(other['City'] == 'Los Angeles')['parks']
parks_lb = other.subset(other['City'] == 'Long Beach')['parks']
parks_a = other.subset(other['City'] == 'Anaheim')['parks']

parks_d = other.subset(other['City'] == 'Dallas')['parks']
parks_atx = other.subset(other['City'] == 'ArlingtonTX')['parks']
parks_fw = other.subset(other['City'] == 'Fort Worth')['parks']

parks_c = other.subset(other['City'] == 'Chicago')['parks']
parks_e = other.subset(other['City'] == 'Elgin')['parks']
parks_n = other.subset(other['City'] == 'Naperville')['parks']

parks_dc = other.subset(other['City'] == 'Washington')['parks']
parks_ava = other.subset(other['City'] == 'ArlingtonVA')['parks']
parks_alex = other.subset(other['City'] == 'Alexandria')['parks']

beaches_la = other.subset(other['City'] == 'Los Angeles')['beaches']
beaches_lb = other.subset(other['City'] == 'Long Beach')['beaches']
beaches_a = other.subset(other['City'] == 'Anaheim')['beaches']

beaches_d = other.subset(other['City'] == 'Dallas')['beaches']
beaches_atx = other.subset(other['City'] == 'ArlingtonTX')['beaches']
beaches_fw = other.subset(other['City'] == 'Fort Worth')['beaches']

beaches_c = other.subset(other['City'] == 'Chicago')['beaches']
beaches_e = other.subset(other['City'] == 'Elgin')['beaches']
beaches_n = other.subset(other['City'] == 'Naperville')['beaches']

beaches_dc = other.subset(other['City'] == 'Washington')['beaches']
beaches_ava = other.subset(other['City'] == 'ArlingtonVA')['beaches']
beaches_alex = other.subset(other['City'] == 'Alexandria')['beaches']

cemetary_la = other.subset(other['City'] == 'Los Angeles')['cemetary']
cemetary_lb = other.subset(other['City'] == 'Long Beach')['cemetary']
cemetary_a = other.subset(other['City'] == 'Anaheim')['cemetary']

cemetary_d = other.subset(other['City'] == 'Dallas')['cemetary']
cemetary_atx = other.subset(other['City'] == 'ArlingtonTX')['cemetary']
cemetary_fw = other.subset(other['City'] == 'Fort Worth')['cemetary']

cemetary_c = other.subset(other['City'] == 'Chicago')['cemetary']
cemetary_e = other.subset(other['City'] == 'Elgin')['cemetary']
cemetary_n = other.subset(other['City'] == 'Naperville')['cemetary']

cemetary_dc = other.subset(other['City'] == 'Washington')['cemetary']
cemetary_ava = other.subset(other['City'] == 'ArlingtonVA')['cemetary']
cemetary_alex = other.subset(other['City'] == 'Alexandria')['cemetary']

shopping_la = other.subset(other['City'] == 'Los Angeles')['shopping']
shopping_lb = other.subset(other['City'] == 'Long Beach')['shopping']
shopping_a = other.subset(other['City'] == 'Anaheim')['shopping']

shopping_d = other.subset(other['City'] == 'Dallas')['shopping']
shopping_atx = other.subset(other['City'] == 'ArlingtonTX')['shopping']
shopping_fw = other.subset(other['City'] == 'Fort Worth')['shopping']

shopping_c = other.subset(other['City'] == 'Chicago')['shopping']
shopping_e = other.subset(other['City'] == 'Elgin')['shopping']
shopping_n = other.subset(other['City'] == 'Naperville')['shopping']

shopping_dc = other.subset(other['City'] == 'Washington')['shopping']
shopping_ava = other.subset(other['City'] == 'ArlingtonVA')['shopping']
shopping_alex = other.subset(other['City'] == 'Alexandria')['shopping']

resturant_la = other.subset(other['City'] == 'Los Angeles')['resturant']
resturant_lb = other.subset(other['City'] == 'Long Beach')['resturant']
resturant_a = other.subset(other['City'] == 'Anaheim')['resturant']

resturant_d = other.subset(other['City'] == 'Dallas')['resturant']
resturant_atx = other.subset(other['City'] == 'ArlingtonTX')['resturant']
resturant_fw = other.subset(other['City'] == 'Fort Worth')['resturant']

resturant_c = other.subset(other['City'] == 'Chicago')['resturant']
resturant_e = other.subset(other['City'] == 'Elgin')['resturant']
resturant_n = other.subset(other['City'] == 'Naperville')['resturant']

resturant_dc = other.subset(other['City'] == 'Washington')['resturant']
resturant_ava = other.subset(other['City'] == 'ArlingtonVA')['resturant']
resturant_alex = other.subset(other['City'] == 'Alexandria')['resturant']

golf_la = other.subset(other['City'] == 'Los Angeles')['golf']
golf_lb = other.subset(other['City'] == 'Long Beach')['golf']
golf_a = other.subset(other['City'] == 'Anaheim')['golf']

golf_d = other.subset(other['City'] == 'Dallas')['golf']
golf_atx = other.subset(other['City'] == 'ArlingtonTX')['golf']
golf_fw = other.subset(other['City'] == 'Fort Worth')['golf']

golf_c = other.subset(other['City'] == 'Chicago')['golf']
golf_e = other.subset(other['City'] == 'Elgin')['golf']
golf_n = other.subset(other['City'] == 'Naperville')['golf']

golf_dc = other.subset(other['City'] == 'Washington')['golf']
golf_ava = other.subset(other['City'] == 'ArlingtonVA')['golf']
golf_alex = other.subset(other['City'] == 'Alexandria')['golf']



In [ ]:
#NORMALIZE HOUSE PRICES ACCORDING TO REGION

#combine cities into metro areas
houses_ca = pd.concat([houses_la, houses_anaheim, houses_longbeach])
houses_tx = pd.concat([houses_dallas, houses_fortworth, houses_arlington_tx])
houses_dc = pd.concat([houses_washington, houses_alexandria, houses_arlington_va])
houses_il = pd.concat([houses_chicago, houses_naperville, houses_elgin])

#convert prices into numpy arrays
price_ca = np.array(houses_ca['Price'])
price_tx = np.array(houses_tx['Price'])
price_dc = np.array(houses_dc['Price'])
price_il = np.array(houses_il['Price'])

#normalize
price_ca = preprocessing.normalize([price_ca])
price_tx = preprocessing.normalize([price_tx])
price_dc = preprocessing.normalize([price_dc])
price_il = preprocessing.normalize([price_il])

#replace prices back into their respective dataframes
houses_ca['Price'] = price_ca
houses_tx['Price'] = price_tx
houses_dc['Price'] = price_dc
houses_il['Price'] = price_il

#subset so that all city dataframes have the normalized price values
houses_la = houses_ca.subset(houses['City'] == 'Los Angeles'.tolower())
houses_anaheim = houses_ca.subset(houses['City'] == 'Anaheim')
houses_longbeach = houses_ca.subset(houses['City'] == 'Long Beach')

houses_dallas = houses_tx.subset(houses['City'] == 'Dallas')
houses_fortworth = houses_tx.subset(houses['City'] == 'Fort Worth')
houses_arlington_tx = houses_tx.subset(houses['City'] == 'Arlington' and houses['State'] == 'TX')

houses_chicago = houses_il.subset(houses['City'] == 'Chicago')
houses_naperville = houses_il.subset(houses['City'] == 'Naperville')
houses_elgin = houses_il.subset(houses['City'] == 'Elgin')

houses_washington = houses_va.subset(houses['City'] == 'Washington')
houses_alexandria = houses_va.subset(houses['City'] == 'Alexandria')
houses_arlington_va = houses_va.subset(houses['City'] == 'Arlington' and houses['State'] == 'VA')

In [ ]:
#ADJUST HOUSE PRICES BY SQUARE FOOTAGE

In [ ]:
#GET DISTANCES

houses_la['dist_hospitals'] = getDist(houses_la, hositals_la)
#...

houses_dallas['dist_gyms'] = getDist(houses_dallas, gyms_dallas)
#...

In [ ]:
#PERFORM ANALYSIS USING DISTANCES AS VARIABLES TO PREDICT PRICE